In [1]:
import numpy as np
import pandas as pd
import unidecode
import os

In [2]:
pd.set_option('display.max_rows',999)
pd.set_option('display.min_rows',10)
pd.set_option('display.float_format', '{:,.2f}'.format)

### Envío Guille 2021-02-23

In [3]:
path = r'C:/Users/Franco/Dropbox/files/properati/'

In [4]:
mylist = []

for file in os.listdir(path):
    if file.endswith(".csv"):
        mylist += [file]

In [5]:
prope = pd.DataFrame()

for file in mylist:
    df = pd.read_csv(path+'{}'.format(file))
    prope = prope.append(df)

In [6]:
prope = prope[prope['l1'].isin(['Argentina', 'Uruguay','Perú', 'Colombia','Ecuador'])]
prope = prope.loc[prope['operation_type']=='Venta',:]
prope = prope[prope['property_type'].isin(['Casa','Departamento','PH','Apartamento'])]

In [7]:
[prope['Country'],prope['City']] = [prope['l1'],prope['l3']]

prope = prope[prope['price'].notna()]
prope = prope[prope['City'].notna()]

Cambio de Buenos Aires y Montevideo

In [8]:
prope = prope.set_index(['l2'])
prope.at['Capital Federal','City'] = 'Ciudad de Buenos Aires'
prope.at['Montevideo','City'] = 'Montevideo'
prope = prope.reset_index()
prope = prope.replace({'Country':{'Perú':'Peru'}})

Otros ajustes

In [9]:
prope = prope.drop(columns=['ad_type','l1','l2','l3','l4','l5','l6','operation_type'])

In [10]:
prope = prope[prope.columns.tolist()[-2:] + [prope.columns.tolist()[-3]] + prope.columns.tolist()[:-3]]

In [11]:
prope = prope.drop(columns=['id','title','description','price_period'])

In [12]:
prope = prope[(prope['price']!=0)]

In [13]:
prope = prope[~((prope['surface_total'].isna())&(prope['surface_covered']).isna())]

In [14]:
prope = prope[((prope['surface_total']!=0)|(prope['surface_covered'])!=0)]

In [15]:
prope = prope.set_index(['Country','City'])

In [16]:
prope['fecha'] = prope['created_on'].str.strip()

#### Tipo de cambio

In [17]:
tc = pd.read_excel(path+'1.1.1.TCM_Serie histórica IQY.xlsx',usecols=[0,1],skiprows=[0,1,2,3,4,5,6]).dropna()
tc['fecha'] = tc[tc.columns[0]].astype(str).str.split(' ').str[0].str.strip()
tc = tc.rename(columns={'Tasa de cambio representativa del mercado (TRM)':'TDC'})

Parto con las df's de prope porq es demasiado grande

In [42]:
prope2 = prope.reset_index().set_index('Country')
prope_no = prope2.drop('Colombia')
prope_no['TDC'] = np.nan
prope_no = prope_no[prope_no['currency']=='USD']
prope_no = prope_no.reset_index()

prope_col = prope2.reset_index()[prope.reset_index()['Country']=='Colombia']
prope_col = prope_col.merge(tc.loc[:,['fecha','TDC']],on='fecha',how='left',indicator=True,validate='m:1')

prope2 = prope_no.append(prope_col.drop(columns='_merge'))

C:\ProgramData\Anaconda3\envs\caf\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [43]:
def f(row):
    if row['currency'] == 'USD':
        val = row['price']
    else:
        val = row['price'] / row['TDC']
    return val

In [44]:
prope2['Precio USD'] = prope2.apply(f,axis=1)

In [45]:
prope2 = prope2[prope2['property_type']!='PH']

In [46]:
def r(row):
    if row['property_type'] in ['Departamento','Apartamento']:
        val = 'Departamento'
    else:
        val = 'Casa'
    return val

In [47]:
prope2['Tipo Propiedad'] = prope2.apply(r,axis=1)

In [48]:
def q05(x):
    return x.quantile(.05)

def q95(x):
    return x.quantile(.95)

In [52]:
med = prope2.loc[:,['Country','Tipo Propiedad','surface_covered']].groupby(['Country','Tipo Propiedad']).agg({'surface_covered':['min',q05,q95, 'max','mean','median','count']})

In [53]:
med = med.droplevel(0,axis=1)

In [59]:
prope2 = prope2.set_index(['Country','Tipo Propiedad']).join(med.loc[:,['q05','q95']])

In [60]:
prope2 = prope2[(prope2['surface_covered']>prope2['q05'])&(prope2['surface_covered']<prope2['q95'])]

In [61]:
prope2 = prope2.reset_index()

In [62]:
prope2['Precio x m2'] = prope2['Precio USD'] / prope2['surface_covered']

In [63]:
prope3 = prope2.groupby(['Country','City','Tipo Propiedad']).median()

In [64]:
prope3['Precio 60m2'] = prope3['Precio x m2'] * 60

In [66]:
prope4 = prope2.groupby(['Country','City']).median()

In [67]:
prope4['Precio 60m2'] = prope4['Precio x m2'] * 60

In [72]:
prope5 = prope4.groupby(['Country','City']).median().drop(columns=['lat','lon','rooms','bedrooms','bathrooms','TDC','q05','q95'])

### Nombres

In [73]:
propef = prope5.reset_index()

In [74]:
cw = pd.read_excel(r'C:\Users\Franco\Dropbox\files\clean\cwpops.xlsx').ffill()

In [75]:
cw['Country'] = cw['Country'].apply(lambda x: unidecode.unidecode(x))
cw['City'] = cw['City'].apply(lambda x: unidecode.unidecode(x))

In [76]:
propef['Country'] = propef['Country'].apply(lambda x: unidecode.unidecode(x))
propef['City'] = propef['City'].apply(lambda x: unidecode.unidecode(x))

In [77]:
#Prope precios de prop's comerciales para obtener nombres limpios
p = pd.read_excel(r'C:\Users\Franco\Dropbox\files\clean\prope\comercial\properati_i.xlsx',index_col=[0,1])
p = p.loc[:,'City Original'].reset_index().drop_duplicates().rename(columns={'City':'City cw','City Original':'City'})
#p = p.reset_index().append(pd.DataFrame([['Argentina','San Miguel de Tucumán','Gran Tucuman-Tafi Viejo']],columns=['Country','City','City cw'])).set_index(['Country','City'])

In [78]:
pgba = pd.DataFrame({'Country':['Argentina'] * 24,
'City' : ['Avellaneda',
'General San Martín',
'Hurlingham',
'Ituzaingó',
'José C.Paz',
'Lanús',
'Lomas de Zamora',
'Malvinas Argentinas',
'Morón',
'Quilmes',
'San Isidro',
'Tres de Febrero',
'Vicente López',
'Almirante Brown',
'Berazategui',
'Esteban Echeverría',
'Ezeiza',
'Florencio Varela',
'La Matanza',
'Merlo',
'Moreno',
'San Fernando',
'San Miguel',
'Tigre'],
'City cw':['Partidos del GBA'] * 24})

In [79]:
otros = pd.DataFrame({'Country':['Argentina','Argentina','Peru','Colombia'],'City':['Ciudad de Buenos Aires','San Miguel de Tucumán','Lima','Bogota D.C'],'City cw':['Ciudad de Buenos Aires','Gran Tucuman-Tafi Viejo','Callao y Lima','Bogota']})

In [80]:
p = p.append(pgba).append(otros)

In [81]:
p['Country'] = p['Country'].apply(lambda x: unidecode.unidecode(x))
p['City'] = p['City'].apply(lambda x: unidecode.unidecode(x))

In [82]:
p = p.set_index(['Country','City']).sort_values(['Country','City'])

In [83]:
propef = propef.set_index(['Country','City'])

In [84]:
prope_cw = propef.join(p,how='inner')

In [85]:
gba = prope_cw[prope_cw['City cw']=='Partidos del GBA']

In [86]:
gba = gba.mean().to_frame().T

In [87]:
[gba['Country'],gba['City'],gba['City cw']] = ['Argentina','Partidos del GBA','Partidos del GBA']

In [88]:
pro = prope_cw[prope_cw['City cw']!='Partidos del GBA']

In [89]:
pro = pro.append(gba.set_index(['Country','City']))

### Sueldos

In [91]:
laboral = pd.read_excel(r'C:/Users/Franco/Dropbox/files/clean/laboral_final(ajustes).xlsx',index_col=[0,1])
cw = pd.read_excel(r'C:/Users/Franco/Dropbox/files/clean/cwpops.xlsx',index_col=[0,1])

In [92]:
salarios = cw.loc[:,'pops'].to_frame().join(laboral.loc[:,'Salario Horario (PPP 2011)'],how='inner')
salarios['Salario Mensual'] = salarios['Salario Horario (PPP 2011)'] * 175

In [93]:
pro = pro.reset_index().rename(columns={'City':'City pro','City cw':'City'}).set_index(['Country','City'])

In [95]:
pro = pro.drop(columns=['price'])

In [97]:
salarios = salarios.join(pro,how='inner')

In [98]:
salarios['Cantidad de salarios mensuales para adquirir Propiedad 60m2'] = salarios['Precio 60m2']/salarios['Salario Mensual']

In [99]:
salarios.to_csv(r'C:/Users/Franco/Dropbox/files/clean/prope/vivienda/salariosxviviendaIII.csv')